# Dog Cat Classifier

### Importing the libraries

In [57]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [59]:
tf.__version__

'2.18.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [64]:
#Using Keras’ ImageDataGenerator to preprocess and augment images for training
#a deep learning model. It rescales pixel values to [0,1], applies transformations
#like shearing, zooming, and horizontal flipping, and
#loads images from the 'dataset/training_set' directory.
#The images are resized to 64x64 pixels, processed in batches of 32, and
#labeled for binary classification.
#This approach helps improve model generalization by
#introducing variations in the training data while efficiently handling large datasets using batch processing.
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [67]:
#Same process as sbove
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [71]:
#Creating a Convolutional Neural Network (CNN) model using TensorFlow/Keras.
cnn = tf.keras.models.Sequential()

#tf.keras.models.Sequential() initializes a sequential model,
#which means layers will be added one after another in a linear stack.

### Step 1 - Convolution

In [74]:
#adding a Convolutional Layer to the CNN model.
#This layer extracts low-level features (like edges and textures) from the input image, helping the CNN recognize patterns for classification.
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

### Step 2 - Pooling

In [77]:
#adding a Max Pooling layer to the CNN model
#Reduces computational complexity and prevents overfitting and helps CNN to focus on the most important features.
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [80]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [83]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [86]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [89]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [93]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [96]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

Epoch 1/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 18s 70ms/step - accuracy: 0.5815 - loss: 0.6735 - val_accuracy: 0.6665 - val_loss: 0.6009
Epoch 2/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 17s 70ms/step - accuracy: 0.6779 - loss: 0.5923 - val_accuracy: 0.7395 - val_loss: 0.5332
Epoch 3/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 17s 68ms/step - accuracy: 0.7392 - loss: 0.5304 - val_accuracy: 0.7510 - val_loss: 0.5084
Epoch 4/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 17s 68ms/step - accuracy: 0.7576 - loss: 0.5052 - val_accuracy: 0.7680 - val_loss: 0.4898
Epoch 5/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 17s 70ms/step - accuracy: 0.7752 - loss: 0.4719 - val_accuracy: 0.7785 - val_loss: 0.4748
Epoch 6/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 19s 77ms/step - accuracy: 0.7796 - loss: 0.4598 - val_accuracy: 0.7695 - val_loss: 0.5032
Epoch 7/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 17s 70ms/step - accuracy: 0.7897 - loss: 0.4479 - val_accuracy: 0.7860 - val_loss: 0.4687
Epoch 8/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 17s 69ms/step - accuracy: 0.8063 - loss: 0.4139 - 

## Part 4 - Making a single prediction

In [102]:
import numpy as np
from tensorflow.keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


In [104]:
print(prediction)

dog
